In [1]:
import inspect
import numpy as np
import pandas as pd
import os
from rapidfuzz import fuzz
from collections.abc import Iterable

In [2]:
help(fuzz.ratio)

Help on cython_function_or_method in module rapidfuzz.fuzz_cpp_avx2:

ratio(s1, s2, *, processor=None, score_cutoff=None)
    Calculates the normalized Indel similarity.
    
    Parameters
    ----------
    s1 : Sequence[Hashable]
        First string to compare.
    s2 : Sequence[Hashable]
        Second string to compare.
    processor: callable, optional
        Optional callable that is used to preprocess the strings before
        comparing them. Default is None, which deactivates this behaviour.
    score_cutoff : float, optional
        Optional argument for a score threshold as a float between 0 and 100.
        For ratio < score_cutoff 0 is returned instead. Default is 0,
        which deactivates this behaviour.
    
    Returns
    -------
    similarity : float
        similarity between s1 and s2 as a float between 0 and 100
    
    See Also
    --------
    rapidfuzz.distance.Indel.normalized_similarity : Normalized Indel similarity
    
    Notes
    -----
    .. imag

# Clean the Oregon dataset

In [3]:
or_monthly_file = os.path.join(os.getcwd(),
                               'data',
                               'input',
                               'OR_Databank_Q1_2007-Q4_2024.csv')
df_or = pd.read_csv(or_monthly_file, low_memory=False)

In [4]:
# check shape for the count of observations and column amount
df_or.shape

(12779, 200)

In [5]:
print(f'60 unique names expected, found {df_or["Hospital Name"].unique().shape[0]}')
# find the most similar names to look for errors
def getSimilarStrPairs(strs, n=None, metric=fuzz.ratio, reverse=False):
    '''
    Get a numpy array (shape=(n, 2)) of pairs from strs ordered most to least similar.
    Parameters:
        strs: An iterable of strings to compare.
        n: If an integer is specified, only the n most similar pairs are returned.
        metric: A function like {f(str1, str2) -> score} to compute string similarity.
                Default is fuzz.ratio from the rapidfuzz package (available via pip or conda-forge)
        reverse: If True, the pairs will be returned in reverse order
    '''
    if not isinstance(strs, Iterable):
        raise TypeError(f"Expected strs to be iterable, got noniterable type {type(strs).__name__}")
    if n and not isinstance(n, int):
        raise TypeError(f'Expected None or integer for n, got {type(n).__name__}')
    if n < 1:
        raise ValueError(f'Expected n to be at least 1, got {n}')
    if not callable(metric):
        raise TypeError(f'Expected metric to be callable, got noncallable type {type(metric).__name__}')
    sig = inspect.signature(metric)
    if len(sig.parameters) < 2:
        raise ValueError(f'Expected metric to take at least 2 parameters, but metric takes {len(sig.parameters)}')
    if not isinstance(reverse, bool):
        raise TypeError(f'Expected reverse to be bool type, got {type(reverse).__name__}')
    np_strs = np.array(strs, dtype=str)
    # indices to match unique pairs from np_strs
    unique_pair_indices = np.triu_indices(np_strs.shape[0], k=1)
    unique_pairs = np.stack([np_strs[unique_pair_indices[0]], np_strs[unique_pair_indices[1]]], axis=1)
    similarities = np.array([metric(str1, str2) for str1, str2 in unique_pairs])
    sort_indices = np.argsort(similarities)
    if not reverse: # change to descending order
        sort_indices = sort_indices[::-1]
    unique_pairs = unique_pairs[sort_indices]
    if n and n < unique_pairs.shape[0]:
        unique_pairs = unique_pairs[:n]
    return unique_pairs

print(getSimilarStrPairs(df_or["Hospital Name"].unique(), n=10))

# standardize whitespace and capitalization in hospital names
df_or['Hospital Name'] = df_or['Hospital Name'].str.replace(r'\s+', ' ', regex=True).str.strip().str.title()
print(f'60 unique names expected, found {df_or["Hospital Name"].unique().shape[0]}')

60 unique names expected, found 62
[['Kaiser Westside Medical  Center' 'Kaiser Westside Medical Center']
 ['PeaceHealth Sacred Heart Medical Center - Riverbend'
  'PeaceHealth Sacred Heart Medical Center - RiverBend']
 ['St. Charles Medical Center - Bend'
  'St. Charles Medical Center - Redmond']
 ['St. Charles Medical Center - Bend'
  'St. Charles Medical Center - Madras']
 ['Harney District Hospital' 'Lake District Hospital']
 ['Saint Alphonsus Medical Center - Baker City'
  'Saint Alphonsus Medical Center - Ontario']
 ['Providence Medford Medical Center'
  'Providence Portland Medical Center']
 ['Kaiser Sunnyside Medical Center' 'Kaiser Westside Medical Center']
 ['Providence Medford Medical Center' 'Providence Newberg Medical Center']
 ['St. Charles Medical Center - Madras'
  'St. Charles Medical Center - Redmond']]
60 unique names expected, found 60


In [6]:
# get the amount of NaNs per column to determine what columns are not helpful
nans = df_or.isna().sum()[df_or.isna().sum() > 0]
nans

Discharges of Acute Inpatient (Commercial)        9911
Discharges of Swing Bed (Commercial)              9911
Discharges of Subacute & LTC (Commercial)         9911
Discharges of DPU (Commercial)                    9911
Total Discharges (Commercial)                     9911
Patient Days of Acute Inpatient (Commercial)      9911
Patient Days of Swing Bed (Commercial)            9911
Patient Days of Subacute & LTC (Commercial)       9911
Patient Days of DPU (Commercial)                  9911
Total Patient Days (Commercial)                   9911
Charges of Acute Inpatient (Commercial)           9911
Charges of Acute Outpatient (Commercial)          9911
Charges of Swing Bed (Commercial)                 9911
Charges of Subacute & LTC (Commercial)            9911
Charges of DPU (Commercial)                       9911
Charges of Home Health (Commercial)               9911
Total Charges (Commercial)                        9911
Contractuals of Acute (Commercial)                9911
Contractua

In [7]:
# calculate the percentage of values that are NaN in each column
nans_percent = (nans / len(df_or)) * 100
nans_percent

Discharges of Acute Inpatient (Commercial)        77.556929
Discharges of Swing Bed (Commercial)              77.556929
Discharges of Subacute & LTC (Commercial)         77.556929
Discharges of DPU (Commercial)                    77.556929
Total Discharges (Commercial)                     77.556929
Patient Days of Acute Inpatient (Commercial)      77.556929
Patient Days of Swing Bed (Commercial)            77.556929
Patient Days of Subacute & LTC (Commercial)       77.556929
Patient Days of DPU (Commercial)                  77.556929
Total Patient Days (Commercial)                   77.556929
Charges of Acute Inpatient (Commercial)           77.556929
Charges of Acute Outpatient (Commercial)          77.556929
Charges of Swing Bed (Commercial)                 77.556929
Charges of Subacute & LTC (Commercial)            77.556929
Charges of DPU (Commercial)                       77.556929
Charges of Home Health (Commercial)               77.556929
Total Charges (Commercial)              

Clearly, since each of these columns are approximately 78% filled with NaNs, we can confidently drop them because imputing these values is not a logical choice. 

In [8]:
# drop the columns that have a heavy amount of NaNs
df_or_dropped_nans = df_or.drop(columns = nans.index)

In [9]:
# check to see how many columns have dropped after the operation
df_or_dropped_nans.shape, len(nans.index) # should be 200 - 51 = 149

((12779, 149), 51)

In [10]:
# get the count of 0's for each column and check to see whether these are valid assignments or not
(df_or_dropped_nans == 0).sum()[(df_or_dropped_nans == 0).sum() > 0]

Critical Access                               7379
Available Beds                                   4
Licensed Beds                                    5
Discharges of Acute Inpatient (Medicaid)       318
Discharges of Acute Inpatient (Self Pay)      2556
Discharges of Acute Inpatient (Others)         839
Discharges of Swing Bed (Medicare)            9196
Discharges of Swing Bed (Medicaid)           11961
Discharges of Swing Bed (Self Pay)           12603
Discharges of Swing Bed (Others)             11746
Discharges of Swing Bed                       8958
Discharges of Subacute & LTC (Medicare)      12669
Discharges of Subacute & LTC (Medicaid)      12375
Discharges of Subacute & LTC (Self Pay)      12424
Discharges of Subacute & LTC (Others)        12720
Discharges of Subacute & LTC                 12219
Discharges of DPU (Medicaid)                 10890
Discharges of DPU (Self Pay)                 11612
Discharges of DPU (Others)                   11006
Total Discharges (Medicaid)    

Goal: We want to understand the meaning behind what 0 is assigned to in the columns to determine if they should be assigned NaN instead.

Solution: Since there is no codebook that tells us about categorical coluumns, we will check the range of the columns that contain 0 to determine if they are categorical or not. Having a small range (e.g., 0 - 5) would maybe indicate a categorical column.

In [11]:
# get the indices of the columns that have zeros
zeros = (df_or_dropped_nans == 0).sum()[(df_or_dropped_nans == 0).sum() > 0].index

# get the ranges of the min and max values to check if they are categorical or not
ranges = df_or_dropped_nans[zeros].agg(['min', 'max'])

# filter for those columns that do not have a high maximum value 
# reason: it is likely that a large maximum value would not represent a categorical column
likely_cat_cols = ranges.columns[ranges.loc['max'] <= 10]

# print out the dataframe containing the possible categorical columns
df_or_dropped_nans[likely_cat_cols]

,Critical Access,Discharges of Swing Bed (Self Pay)
0,False,0
1,False,0
2,False,0
3,False,0
4,False,0
...,...,...
12774,False,0
12775,False,0
12776,False,0
12777,False,0


Since the first column is clearly a binary value, 0 is meaningful and therefore it needs to be kept instead of being assigned a NaN. For the other column (Discharges of Swing Bed (Self Pay)), it is claimed in the Databank Field List that this column represents "Discharges from certified swing bed care. Self-pay payers." This means that 0 has a relevant meaning because it refers to "There are no discharges from self-paid payers in swing beds" and should therefore not be assigned NaN.

In [12]:
# want to now check the types and deal with objects 
df_or_dropped_nans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12779 entries, 0 to 12778
Columns: 149 entries, AHA ID to Total Revenue
dtypes: bool(1), int64(43), object(105)
memory usage: 14.4+ MB


In [13]:
# get only the columns containing objects
object_cols = df_or_dropped_nans.select_dtypes(include='object').columns

# remove the columns that are definitely not numeric
object_cols_wo_cat = object_cols.drop(['Hospital Name', 'Hospital Short Name', 'Type'])

In [14]:
# loop through the object type columns and determine what can and can't be converted to numeric without coercing
for column in object_cols_wo_cat:
    try:
        pd.to_numeric(df_or_dropped_nans[column])
    except Exception as e:
        print(f"{column:<45}: {e}")

Discharges of Acute Inpatient (Medicare)     : Unable to parse string "1,098" at position 6401
Discharges of Acute Inpatient                : Unable to parse string "1,233" at position 864
Discharges of DPU (Medicare)                 : Unable to parse string "-3,164" at position 7254
Discharges of DPU                            : Unable to parse string "-3,204" at position 7254
Total Discharges (Medicare)                  : Unable to parse string "1,140" at position 6401
Total Discharges                             : Unable to parse string "1,313" at position 864
Patient Days of Acute Inpatient (Medicare)   : Unable to parse string "2,003" at position 216
Patient Days of Acute Inpatient (Medicaid)   : Unable to parse string "2,471" at position 864
Patient Days of Acute Inpatient (Self Pay)   : Unable to parse string "1,106" at position 3456
Patient Days of Acute Inpatient (Others)     : Unable to parse string "1,676" at position 5321
Patient Days of Acute Inpatient              : Unabl

In [15]:
# loop through the columns that are object type and not categorical replace the ',' with ''
for column in df_or_dropped_nans[object_cols_wo_cat]:
    df_or_dropped_nans[column] = df_or_dropped_nans[column].str.replace(',', '', regex=False)

In [16]:
# try the loop again (also w/o categorical cols) to check if replacing ',' with '' resolves the errors
for column in df_or_dropped_nans[object_cols_wo_cat].columns:
    try:
        pd.to_numeric(df_or_dropped_nans[column])
    except Exception as e:
        print(f"{column:<45}: {e}")

In [17]:
# there are no errors from the loop, therefore we can now add coercion to properly convert to numeric
for column in df_or_dropped_nans[object_cols_wo_cat].columns:
    df_or_dropped_nans[column] = pd.to_numeric(df_or_dropped_nans[column], errors='coerce')

In [18]:
df_or_dropped_nans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12779 entries, 0 to 12778
Columns: 149 entries, AHA ID to Total Revenue
dtypes: bool(1), int64(145), object(3)
memory usage: 14.4+ MB


Since there are 3 categorical columns and three object type columns left, we can confidently conclude that the object types have been properly handled.

In [19]:
# set up data for aggregation
categorical_cols = [
    'Hospital Name',
    'Hospital Short Name',
    'Type',
    'Critical Access',
] # excluding AHA Id because its a grouping column
proportion_cols = [
    r'Operating Margin %',
    r'total margin %',
    r'CCR',
    r'charity care %',
    r'bad debt %',
    r'Medicaid %',
    r'Medicare %',
    r'Commercial %',
    r'Self Pay %',
]
bed_cols = list(df_or_dropped_nans.filter(regex='(?i)beds').columns)
sum_agg_cols = list(set(df_or_dropped_nans.columns).difference(set(bed_cols+categorical_cols+proportion_cols+['Month', 'Year', 'Quarter', 'AHA ID'])))

In [20]:
# aggregate data
agg_dict = {key:'sum' for key in sum_agg_cols}
agg_dict.update({key:(lambda s: s.mode().iloc[0]) for key in categorical_cols})
agg_dict.update({key:'last' for key in bed_cols})
# agg_dict.update({key:'mean' for key in proportion_cols})

quarterly_sums = df_or_dropped_nans.sort_values('Month')\
    .drop(columns=['Month'])\
    .groupby(['AHA ID', 'Year', 'Quarter']).agg(agg_dict).reset_index()

quarterly_sums.head()

,AHA ID,Year,Quarter,Total Patient Days (Self Pay),Total Contractual Allowances (Medicaid),Patient Days of DPU,Uncompensated Care,Gross Patient Accounts Receivable (Others),Emergency Department Visits,Patient Days of Acute Inpatient (Self Pay),...,Discharges of Acute Inpatient (Others),Charges of Swing Bed (Others),Charges of Acute Inpatient (Others),Charges of Subacute & LTC,Hospital Name,Hospital Short Name,Type,Critical Access,Available Beds,Licensed Beds
0,6920003,2007,1,1932,33963338,2702,20221600,281437663,11787,1831,...,1930,0,57852754,0,Legacy Emanuel Medical Center,Legacy Emanuel Med Ctr,DRG,False,401,554
1,6920003,2007,2,1968,39446639,2894,24867198,286899220,11178,1907,...,2071,0,67123379,0,Legacy Emanuel Medical Center,Legacy Emanuel Med Ctr,DRG,False,400,554
2,6920003,2007,3,2228,33742710,2706,27001412,312444180,10946,2116,...,2148,0,72086363,0,Legacy Emanuel Medical Center,Legacy Emanuel Med Ctr,DRG,False,398,554
3,6920003,2007,4,1642,30356389,2669,19238363,307649411,10815,1520,...,1983,0,65833393,0,Legacy Emanuel Medical Center,Legacy Emanuel Med Ctr,DRG,False,410,554
4,6920003,2008,1,1495,36108352,2695,16161666,295498881,12166,1364,...,2052,0,68253257,0,Legacy Emanuel Medical Center,Legacy Emanuel Med Ctr,DRG,False,412,554


In [21]:
quarterly_sums = quarterly_sums[list(df_or_dropped_nans.columns)[:5] + list(df_or_dropped_nans.columns)[6:]]
quarterly_sums.head(1)

,AHA ID,Hospital Name,Hospital Short Name,Type,Critical Access,Quarter,Year,Available Beds,Licensed Beds,Discharges of Acute Inpatient (Medicare),...,Total Margin,Gross Patient Accounts Receivable (Medicare),Gross Patient Accounts Receivable (Medicaid),Gross Patient Accounts Receivable (Self Pay),Gross Patient Accounts Receivable (Others),Gross Patient Accounts Receivable,Uncompensated Care,Inpatient Discharges,Total Operating Revenue,Total Revenue
0,6920003,Legacy Emanuel Medical Center,Legacy Emanuel Med Ctr,DRG,False,1,2007,401,554,771,...,706200,50749312,75957402,54907221,281437663,463051598,20221600,4663,98490357,100194315


In [22]:
# set the finalized oregon dataframe to a readable variable name
oregon = quarterly_sums

In [23]:
oregon.shape

(4260, 148)

# Clean the Washington dataset

In [24]:
wa_file = os.path.join(os.getcwd(),
                       'data',
                       'input',
                       'WA_Hospital_Financial_Quarterly_Aggregate_Report_20250520.csv')
df_wa = pd.read_csv(wa_file)

In [25]:
df_wa.shape

(2840, 163)

In [26]:
# get the amount of NaNs in each column
nans_wa = df_wa.isna().sum()[df_wa.isna().sum() > 0]
nans_wa

Professional_Fee_Expense    1880
dtype: int64

In [27]:
# check the ratio of NaNs for Professional_Fee_Expense
1880 / df_wa.shape[0] * 100

66.19718309859155

In [28]:
# drop the column because there is a significant amount of NaNs for it
df_wa.drop('Professional_Fee_Expense', axis = 1, inplace = True)

In [29]:
print(f'Unique names count (uncleaned): {df_wa["Hospital_Name"].unique().shape[0]}')
print(getSimilarStrPairs(df_wa['Hospital_Name'].unique(), n=10))

print(df_wa[['Hospital_Name', 'City', 'License_Number']]\
      .loc[df_wa['Hospital_Name'].isin(['Samaritan Hospital', 'Good Samaritan Hospital'])])
print(df_wa[['Hospital_Name', 'City', 'License_Number']]\
      .loc[df_wa['Hospital_Name'].isin(['Swedish Health Services Dba Swedish Medical Cent', 'Swedish Health Services Dba Swedish Medical Cente'])])

# Special mappings for washington
wa_name_mappings = {
    'Swedish Health Services DBA Swedish Medical Cent': 'Swedish Health Services DBA Swedish Medical Center',
    'Swedish Health Services DBA Swedish Medical Cente': 'Swedish Health Services DBA Swedish Medical Center'
}
# standardize whitespace and capitalization in hospital names
df_wa['Hospital_Name'] = df_wa['Hospital_Name']\
    .replace(wa_name_mappings)\
    .str.replace(r'\s+', ' ', regex=True).str.strip().str.title()
print(f'Unique names count (cleaned): {df_wa["Hospital_Name"].unique().shape[0]}')

Unique names count (uncleaned): 106
[['Swedish Health Services DBA Swedish Medical Cent'
  'Swedish Health Services DBA Swedish Medical Cente']
 ['PeaceHealth St John Medical Center'
  'PeaceHealth St Joseph Medical Center']
 ['BHC Fairfax Behavioral Health - Everett'
  'BHC Fairfax Behavioral Health - Monroe']
 ['Samaritan Hospital' 'Good Samaritan Hospital']
 ['Valley Medical Center' 'Quincy Valley Medical Center']
 ['PeaceHealth St John Medical Center'
  'PeaceHealth Southwest Medical Center']
 ['PeaceHealth St Joseph Medical Center'
  'PeaceHealth Southwest Medical Center']
 ['Astria Regional Medical Center' 'Kadlec Regional Medical Center']
 ['Klickitat County Public Hospital District #1'
  'Columbia County Public Hospital District No. 1']
 ['Public Hospital District No 1 of Mason County WA'
  'Public Hospital District #1-A of Whitman County']]
                Hospital_Name        City  License_Number
33         Samaritan Hospital  Moses Lake              78
36    Good Samaritan H

In [30]:
# check to see the count of zeros in each column
(df_wa == 0).sum()[(df_wa == 0).sum() > 0]

Licensed_Beds                            168
Acute_Care_Medicare_Discharges           363
Acute_Care_Medicaid_Discharges           409
Acute_Care_Self_Pay_Discharges          2040
Acute_Care_Commercial_Discharges        2053
                                        ... 
Gross_Accounts_Receivable_Self_Pay      1913
Gross_Accounts_Receivable_Commercial    2110
Gross_Accounts_Receivable_Other          345
Total_Gross_Accounts_Receivable          278
Casemix Index                             85
Length: 157, dtype: int64

In [31]:
# get the indices of the columns that have zeros
zeros_wa = (df_wa == 0).sum()[(df_wa == 0).sum() > 0].index

# get the ranges of the min and max values to check if they are categorical or not
ranges_wa = df_wa[zeros_wa].agg(['min', 'max'])

# filter for those columns that do not have a high maximum value 
# reason: it is likely that a large maximum value would not represent a categorical column
likely_cat_cols_wa = ranges_wa.columns[ranges_wa.loc['max'] <= 10]

# print out the dataframe containing the possible categorical columns
df_wa[likely_cat_cols_wa]

,SNF_Self_Pay_Discharges,SNF_Commercial_Discharges,Casemix Index
0,0,0,2.1068
1,0,0,0.6334
2,0,0,2.1873
3,0,0,0.6585
4,0,0,0.7591
...,...,...,...
2835,0,0,0.0000
2836,0,0,0.0000
2837,0,0,0.0000
2838,0,0,0.0000


The three columns are all not categorical, therefore, we can likely conclude that there are no columns where a 0 means something other than, for example, 0 self pay discharges. We can now confidently proceed and treat 0's as meaningful.

In [32]:
# check to see the datatypes of the columns
# reason: want to know if any conversions are needed
df_wa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2840 entries, 0 to 2839
Columns: 162 entries, License_Number to Casemix Index
dtypes: float64(1), int64(159), object(2)
memory usage: 3.5+ MB


In [33]:
# check the columns that are of object dtype
df_wa.select_dtypes(include='object').columns

Index(['Hospital_Name', 'City'], dtype='object')

These are appropriately of 'object' datatype, therefore, we can continue with cleaning.

### Feature variable matching

In [34]:
# rename the column due to a typo in the dataset
df_wa = df_wa.rename(columns={'Psych_Rehab_DCU_Total_Revenue': 'Psych_Rehab_CDU_Total_Revenue'})

In [35]:
# Need a column mapping for the columns in the Oregon dataset to concatenate with the Washington dataset
or_to_wa_rename = {
    'AHA ID': 'License_Number',
    'Hospital Name': 'Hospital_Name',
    'Year': 'Year',
    'Quarter': 'Quarter',
    'Licensed Beds': 'Licensed_Beds',

    # Acute Inpatient Discharges
    'Discharges of Acute Inpatient (Medicare)': 'Acute_Care_Medicare_Discharges',
    'Discharges of Acute Inpatient (Medicaid)': 'Acute_Care_Medicaid_Discharges',
    'Discharges of Acute Inpatient (Self Pay)': 'Acute_Care_Self_Pay_Discharges',
    'Discharges of Acute Inpatient (Others)': 'Acute_Care_Other_Discharges',
    'Discharges of Acute Inpatient': 'Acute_Care_Total_Discharges',

    # Swing Bed Discharges
    'Discharges of Swing Bed (Medicare)': 'Swing_Bed_Medicare_Discharges',
    'Discharges of Swing Bed (Medicaid)': 'Swing_Bed_Medicaid_Discharges',
    'Discharges of Swing Bed (Self Pay)': 'Swing_Bed_Self_Pay_Discharges',
    'Discharges of Swing Bed (Others)': 'Swing_Bed_Other_Discharges',
    'Discharges of Swing Bed': 'Swing_Bed_Total_Discharges',

    # Subacute & LTC (likely SNF in WA)
    'Discharges of Subacute & LTC (Medicare)': 'SNF_Medicare_Discharges',
    'Discharges of Subacute & LTC (Medicaid)': 'SNF_Medicaid_Discharges',
    'Discharges of Subacute & LTC (Self Pay)': 'SNF_Self_Pay_Discharges',
    'Discharges of Subacute & LTC (Others)': 'SNF_Other_Discharges',
    'Discharges of Subacute & LTC': 'SNF_Total_Discharges',

    # DPU/psych (Psych_Rehab_CDU in WA)
    'Discharges of DPU (Medicare)': 'Psych_Rehab_CDU_Medicare_Discharges',
    'Discharges of DPU (Medicaid)': 'Psych_Rehab_CDU_Medicaid_Discharges',
    'Discharges of DPU (Self Pay)': 'Psych_Rehab_CDU_Self_Pay_Discharges',
    'Discharges of DPU (Others)':  'Psych_Rehab_CDU_Other_Discharges',
    'Discharges of DPU': 'Psych_Rehab_CDU_Total_Discharges',

    # Total Discharges (by payer)
    'Total Discharges (Medicare)': 'Total_Medicare_Discharges',
    'Total Discharges (Medicaid)': 'Total_Medicaid_Discharges',
    'Total Discharges (Self Pay)': 'Total_Self_Pay_Discharges',
    'Total Discharges (Others)':  'Total_Other_Discharges',
    'Total Discharges': 'Total_Discharges',

    # Patient Days Acute
    'Patient Days of Acute Inpatient (Medicare)': 'Acute_Care_Medicare_Patient_Days',
    'Patient Days of Acute Inpatient (Medicaid)': 'Acute_Care_Medicaid_Patient_Days',
    'Patient Days of Acute Inpatient (Self Pay)': 'Acute_Care_Self_Pay_Patient_Days',
    'Patient Days of Acute Inpatient (Others)': 'Acute_Care_Other_Patient_Days',
    'Patient Days of Acute Inpatient': 'Acute_Care_Total_Patient_Days',

    # Swing Bed Patient Days
    'Patient Days of Swing Bed (Medicare)': 'Swing_Bed_Medicare_Patient_Days',
    'Patient Days of Swing Bed (Medicaid)': 'Swing_Bed_Medicaid_Patient_Days',
    'Patient Days of Swing Bed (Self Pay)': 'Swing_Bed_Self_Pay_Patient_Days',
    'Patient Days of Swing Bed (Others)': 'Swing_Bed_Other_Patient_Days',
    'Patient Days of Swing Bed': 'Swing_Bed_Total_Patient_Days',

    # Subacute & LTC Patient Days (SNF)
    'Patient Days of Subacute & LTC (Medicare)': 'SNF_Medicare_Patient_Days',
    'Patient Days of Subacute & LTC (Medicaid)': 'SNF_Medicaid_Patient_Days',
    'Patient Days of Subacute & LTC (Self Pay)': 'SNF_Self_Pay_Patient_Days',
    'Patient Days of Subacute & LTC (Others)': 'SNF_Other_Patient_Days',
    'Patient Days of Subacute & LTC': 'SNF_Total_Patient_Days',

    # DPU/Psych (Psych_Rehab_CDU in WA)
    'Patient Days of DPU (Medicare)': 'Psych_Rehab_CDU_Medicare_Patient_Days',
    'Patient Days of DPU (Medicaid)': 'Psych_Rehab_CDU_Medicaid_Patient_Days',
    'Patient Days of DPU (Self Pay)': 'Psych_Rehab_CDU_Self_Pay_Patient_Days',
    'Patient Days of DPU (Others)': 'Psych_Rehab_CDU_Other_Patient_Days',
    'Patient Days of DPU': 'Psych_Rehab_CDU_Total_Patient_Days',

    # Total Patient Days
    'Total Patient Days (Medicare)': 'Total_Medicare_Patient_Days',
    'Total Patient Days (Medicaid)': 'Total_Medicaid_Patient_Days',
    'Total Patient Days (Self Pay)': 'Total_Self_Pay_Patient_Days',
    'Total Patient Days (Others)': 'Total_Other_Patient_Days',
    'Total Patient Days': 'Total_Patient_Days',

    # Surgeries, births, admissions, visits
    'Inpatient Surgeries': 'Total_Inpatient_Surgeries',
    'Births': 'Total_Births',
    'Newborn Patient Days': 'Total_Newborn_Days',
    'Admissions from ED': 'Number_of_Admissions_from_ER',
    'Emergency Department Visits': 'Emergency_Room_Visits',
    'Ambulatory Surgery Visits': 'Outpatient_Surgery_Visits',
    'Observation Visits': 'Observation_Visits',
    'Home Health Visits': 'Home_Health_Visits',
    'Other Outpatient Visits': 'All_Other_Visits',
    'Total Outpatient Visits': 'Total_Outpatient_Visits',

    # Acute inpatient charges (revenue)
    'Charges of Acute Inpatient (Medicare)': 'Acute_Medicare_Inpatient_Revenue',
    'Charges of Acute Inpatient (Medicaid)': 'Acute_Medicaid_Inpatient_Revenue',
    'Charges of Acute Inpatient (Self Pay)': 'Acute_Self_Pay_Inpatient_Revenue',
    'Charges of Acute Inpatient (Others)': 'Acute_Other_Inpatient_Revenue',
    'Charges of Acute Inpatient': 'Acute_Total_Inpatient_Revenue',

    # Acute outpatient charges (revenue)
    'Charges of Acute Outpatient (Medicare)': 'Acute_Medicare_Outpatient_Revenue',
    'Charges of Acute Outpatient (Medicaid)': 'Acute_Medicaid_Outpatient_Revenue',
    'Charges of Acute Outpatient (Self Pay)': 'Acute_Self_Pay_Outpatient_Revenue',
    'Charges of Acute Outpatient (Others)': 'Acute_Other_Outpatient_Revenue',
    'Charges of Acute Outpatient': 'Acute_Total_Outpatient_Revenue',

    # Swing bed charges (revenue)
    'Charges of Swing Bed (Medicare)': 'Swing_Bed_Medicare_Revenue',
    'Charges of Swing Bed (Medicaid)': 'Swing_Bed_Medicaid_Revenue',
    'Charges of Swing Bed (Self Pay)': 'Swing_Bed_Self_Pay_Revenue',
    'Charges of Swing Bed (Others)': 'Swing_Bed_Other_Revenue',
    'Charges of Swing Bed': 'Swing_Bed_Total_Revenue',

    # SNF (Subacute & LTC) charges (revenue)
    'Charges of Subacute & LTC (Medicare)': 'SNF_Medicare_Revenue',
    'Charges of Subacute & LTC (Medicaid)': 'SNF_Medicaid_Revenue',
    'Charges of Subacute & LTC (Self Pay)': 'SNF_Self_Pay_Revenue',
    'Charges of Subacute & LTC (Others)': 'SNF_Other_Revenue',
    'Charges of Subacute & LTC': 'SNF_Total_Revenue',

    # DPU charges (psych rehab revenue)
    'Charges of DPU (Medicare)': 'Psych_Rehab_CDU_Medicare_Revenue',
    'Charges of DPU (Medicaid)': 'Psych_Rehab_CDU_Medicaid_Revenue',
    'Charges of DPU (Self Pay)': 'Psych_Rehab_CDU_Self_Pay_Revenue',
    'Charges of DPU (Others)': 'Psych_Rehab_CDU_Other_Revenue',
    'Charges of DPU': 'Psych_Rehab_CDU_Total_Revenue',

    # Home Health Revenue
    'Charges of Home Health (Medicare)': 'Home_Health_Medicare_Revenue',
    'Charges of Home Health (Medicaid)': 'Home_Health_Medicaid_Revenue',
    'Charges of Home Health (Self Pay)': 'Home_Health_Self_Pay_Revenue',
    'Charges of Home Health (Others)': 'Home_Health_Other_Revenue', 
    'Charges of Home Health': 'Home_Health_Total_Revenue',

    'Contractuals of Acute (Medicare)':     'Acute_Contractuals_Medicare',
    'Contractuals of Acute (Medicaid)':     'Acute_Contractuals_Medicaid',
    'Contractuals of Acute (Self Pay)':     'Acute_Contractuals_Self_Pay',
    'Contractuals of Acute (Others)':       'Acute_Contractuals_Other',
    'Contractuals of Acute':                'Acute_Contractuals_Total',

    'Contractuals of Other (Medicare)':     'Other_Contractuals_Medicare',
    'Contractuals of Other (Medicaid)':     'Other_Contractuals_Medicaid',
    'Contractuals of Other (Self Pay)':     'Other_Contractuals_Self_Pay',
    'Contractuals of Other (Others)':       'Other_Contractuals_Other',
    'Contractuals of Other':                'Other_Contractuals_Total',

    'Total Contractual Allowances (Medicare)': 'Total_Medicare_Contractuals',
    'Total Contractual Allowances (Medicaid)': 'Total_Medicaid_Contractuals',
    'Total Contractual Allowances (Self Pay)': 'Total_Self_Pay_Contractuals',
    'Total Contractual Allowances (Others)':   'Total_Other_Contractuals',
    'Total Contractual Allowances':            'Total_Contractuals',

    
    # Payroll and hours
    'Facility Payroll Amount': 'Facility_Payroll',
    'Facility Payroll Hours': 'Facility_Paid_Hours',
    'Physician Payroll Amount': 'Physician_Payroll',
    'Physician Payroll Hours': 'Physician_Paid_Hours',
    'Total Payroll Amount': 'Total_Payroll',
    'Total Payroll Hours': 'Total_Paid_Hours',

    # Expenses and other summary fields
    'Benefit Expense': 'Employee_Benefit_Expenses',
    'Supply Expense': 'Supply_Expenses',
    'Depreciation Expense': 'Depreciation_Expense',
    'Interest Expense': 'Interest_Expenses',
    'Bad Debt': 'Bad_Debts',
    'Other Expense': 'All_Other_Expenses',
    'Total Operating Expense': 'Total_Operating_Expenses',
    'Other Operating Revenue': 'Other_Operation_Revenue',
    'Operating Margin': 'Operating_Margin',
    'Net Nonoperating Gains': 'Net_Non_Operating_Gains_Losses',
    'Tax Subsidies ': 'Tax_Subsidies',
    'Total Margin ': 'Gross_Total_Margin',

    # Receivables
    'Gross Patient Accounts Receivable (Medicare)': 'Gross_Accounts_Receivable_Medicare',
    'Gross Patient Accounts Receivable (Medicaid)': 'Gross_Accounts_Receivable_Medicaid',
    'Gross Patient Accounts Receivable (Self Pay)': 'Gross_Accounts_Receivable_Self_Pay',
    'Gross Patient Accounts Receivable (Others)': 'Gross_Accounts_Receivable_Other',
    'Gross Patient Accounts Receivable': 'Total_Gross_Accounts_Receivable',

    # Charity/Uncompensated
    'Charity Care': 'Total_Charity_Care',
    # 'Uncompensated Care':  # Not found in WA columns

    # Final summaries
    'Total Revenue': 'Total_Revenue'

    
}

In [36]:
# rename the oregon columns
oregon_renamed = oregon.rename(columns = or_to_wa_rename)

In [37]:
# drop columns in the oregon dataset that do not combine seamlessly with the washington columns
cols_to_drop = ['Total Charges (Medicare)', 'Total Charges (Medicaid)', 'Total Charges (Self Pay)', 
                'Total Charges (Others)', 'Total Charges', 'Uncompensated Care', 'Inpatient Discharges',
                'Total Operating Revenue', 'Hospital Short Name', 'Type', 'Critical Access', 
                'Available Beds', 'Net Patient Revenue']

oregon_renamed.drop(cols_to_drop, axis = 1, inplace = True)

In [38]:
# drop columns in the washington dataset that are not found in the oregon dataset

cols_to_drop_wa = ['Total_Medicare_Revenue', 'Total_Medicaid_Revenue', 'Total_Self_Pay_Revenue', 'Total_Other_Revenue',
                  'City', 'Casemix Index']
commercial_cols = [col for col in df_wa.columns if "Commercial" in col]

df_wa.drop(commercial_cols + cols_to_drop_wa, axis=1, inplace=True)

In [39]:
# check shape to see if column number matches
df_wa.shape[1], oregon_renamed.shape[1]

(135, 135)

In [40]:
# assign the state to each dataset as a new column
df_wa['State'] = 'Washington'
oregon_renamed['State'] = 'Oregon'

### Combine datasets

In [41]:
# are any names shared?
shared_names = list(set(df_wa['Hospital_Name'].unique().tolist())\
                    .intersection(set(oregon_renamed['Hospital_Name'].unique().tolist())))
print(shared_names)

# there is one shared name, so for easier use/readability, rename both include the state
oregon_renamed['Hospital_Name'] = oregon_renamed['Hospital_Name'].replace({'St. Anthony Hospital': 'St. Anthony Hospital - OR'})
df_wa['Hospital_Name'] = df_wa['Hospital_Name'].replace({'St. Anthony Hospital': 'St. Anthony Hospital - WA'})

['St. Anthony Hospital']


In [42]:
df = pd.concat([df_wa, oregon_renamed])
print(f'Unique names count (60 + 105 = 165 expected): {df["Hospital_Name"].unique().shape[0]}')

Unique names count (60 + 105 = 165 expected): 165


### Create the target variable: increase in percentage in total revenue year-over-year by quarter

In [43]:
# sort by year, for easy calculation using shift ('State', 'Hospital_Name', 'Quarter' part of groupby) 
df = df.sort_values(['Year'])

In [44]:
# get next year same quarter revenue for each hospital
df['Next_Year_Total_Revenue'] = df.groupby(['State', 'Hospital_Name', 'Quarter'])['Total_Revenue'].shift(-1)

C:\Users\ellin\AppData\Local\Temp\ipykernel_6624\2090024818.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['Next_Year_Total_Revenue'] = df.groupby(['State', 'Hospital_Name', 'Quarter'])['Total_Revenue'].shift(-1)


In [45]:
# check nans
print(f'Next Year Missing: {df["Next_Year_Total_Revenue"].isna().sum()}')
print(f'This Year Missing: {df["Total_Revenue"].isna().sum()}')
print(f'Number of rows with 0 total revenue: {(df["Total_Revenue"] == 0).sum()}')

# How many of the rows with 0 total revenue overlap with the final record for a (hosp, quarter)
zero_revenue_rows = df[df['Total_Revenue'] == 0]
last_records_for_hospital_quarter = df.sort_values(['Hospital_Name', 'Quarter', 'Year'])\
                 .groupby(['Hospital_Name', 'Quarter'], as_index=False)\
                 .last()
overlap_count = len(pd.merge(zero_revenue_rows, last_records_for_hospital_quarter,
                             on=['Hospital_Name', 'Quarter', 'Year']))
print("Number of overlapping rows:", overlap_count)

zero_revenue_rows = df[np.isclose(df['Total_Revenue'], 0)]
last_records_for_hospital_quarter = df.loc[df["Next_Year_Total_Revenue"].isna()]
overlap_count = len(pd.merge(zero_revenue_rows, last_records_for_hospital_quarter,
                             on=['Hospital_Name', 'Quarter', 'Year']))
print("Number of overlapping rows:", overlap_count)

Next Year Missing: 660
This Year Missing: 0
Number of rows with 0 total revenue: 222
Number of overlapping rows: 24
Number of overlapping rows: 24


C:\Users\ellin\AppData\Local\Temp\ipykernel_6624\1477888455.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  last_records_for_hospital_quarter = df.sort_values(['Hospital_Name', 'Quarter', 'Year'])\
C:\Users\ellin\AppData\Local\Temp\ipykernel_6624\1477888455.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  last_records_for_hospital_quarter = df.sort_values(['Hospital_Name', 'Quarter', 'Year'])\


In [46]:
# calculate the percentage increase
df['Pct_Inc_Total_Revenue_Next_Year'] = (
    (df['Next_Year_Total_Revenue'] - df['Total_Revenue']) /
    df['Total_Revenue']
) * 100

# year_diff for ensuring only consecutive years are compared
year_diff = df.groupby(['State', 'Hospital_Name', 'Quarter'])['Year'].diff().shift(-1)

df['Pct_Inc_Total_Revenue_Next_Year'] = np.where(year_diff == 1,
                                                 df['Pct_Inc_Total_Revenue_Next_Year'], np.nan)
print(f'Pct Increase Missing: {df["Pct_Inc_Total_Revenue_Next_Year"].isna().sum()}')
# round for ease of reading
#df_wa['Pct_Inc_Total_Revenue_from_PrevYearQuarter'] = df_wa['Pct_Inc_Total_Revenue_from_PrevYearQuarter'].round(2)

Pct Increase Missing: 1453


C:\Users\ellin\AppData\Local\Temp\ipykernel_6624\4290287913.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['Pct_Inc_Total_Revenue_Next_Year'] = (


In [47]:
df = df.sort_values(['Hospital_Name', 'Year', 'Quarter']).reset_index(drop=True).copy()

In [48]:
df.head()

,License_Number,Hospital_Name,Year,Quarter,Licensed_Beds,Acute_Care_Medicare_Discharges,Acute_Care_Medicaid_Discharges,Acute_Care_Self_Pay_Discharges,Acute_Care_Other_Discharges,Acute_Care_Total_Discharges,...,Tax_Subsidies,Gross_Total_Margin,Gross_Accounts_Receivable_Medicare,Gross_Accounts_Receivable_Medicaid,Gross_Accounts_Receivable_Self_Pay,Gross_Accounts_Receivable_Other,Total_Gross_Accounts_Receivable,State,Next_Year_Total_Revenue,Pct_Inc_Total_Revenue_Next_Year
0,6920770,Adventist Health Columbia Gorge Medical Center,2007,1,49,222,133,30,225,610,...,0,209837,17608069,6151543,9750249,15949380,49459241,Oregon,16830885.0,NaN
1,6920770,Adventist Health Columbia Gorge Medical Center,2007,2,49,222,119,48,199,588,...,0,744077,20506477,5967128,10114763,17223231,53811599,Oregon,16320272.0,NaN
2,6920770,Adventist Health Columbia Gorge Medical Center,2007,3,49,195,138,37,250,620,...,0,299308,20999405,4993956,10996618,21216575,58206554,Oregon,16414247.0,NaN
3,6920770,Adventist Health Columbia Gorge Medical Center,2007,4,49,195,123,53,205,576,...,0,-294429,21545389,5862029,10127498,20336947,57871863,Oregon,17453505.0,NaN
4,6920770,Adventist Health Columbia Gorge Medical Center,2008,1,49,227,156,44,237,664,...,0,1139743,21630299,5695074,12004199,22519231,61848803,Oregon,17964144.0,6.733211


In [49]:
out_path = os.path.join(os.getcwd(),
                        'data',
                        'oregon_washington_combined_data.csv')
df.to_csv(out_path, index=False)

In [50]:
target_na_count = df['Pct_Inc_Total_Revenue_Next_Year'].isna().sum()
print('Expecting 165x4=660 NAs for 165 hospitals and 4 quarters missing each')
print(f'Actual NA count: {target_na_count}')
print(f'NA percent: {100 * target_na_count / df.shape[0]}%')

Expecting 165x4=660 NAs for 165 hospitals and 4 quarters missing each
Actual NA count: 1453
NA percent: 20.464788732394368%
